In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import matplotlib.pyplot as plt
import math
import cv2

In [ ]:
gray_lena_img = cv2.imread('/content/drive/MyDrive/Colab Notebooks/cv_2024_1/lena.bmp', cv2.IMREAD_GRAYSCALE) #이미지를 흑백이미지로 읽음
plt.imshow(gray_lena_img, cmap='gray') #흑백의 원본 사진 출력


In [ ]:
# 1) Design a function to generate Gaussian image pyramid.
# Input : image, scaling factor (0<s<1), number of levels
# Output : image pyramid

#가우시안 필터 생성
def gaussian_filter(k,sig):
    ax = np.linspace(-(k//2), k//2, k)
    x, y = np.meshgrid(ax, ax)
    kernel = np.exp(-(x**2 + y**2) / (2 * sig**2)) # k x k 크기의 커널 생성
    return kernel / np.sum(kernel) # 정규화. 전체 필터의 요소들의 합은 1

gaussian_filter_mask = gaussian_filter(3,1.5)

# 컨볼루션 진행 함수
def convolution(image, filter):

  image_height, image_width = image.shape
  filter_height, filter_width = filter.shape
  padding = filter_height // 2
  filtered_image = np.zeros_like(image) #새로운 복사 이미지

  #반복문을 수행하면서 패딩&컨볼루션 진행
  for y in range(padding, image_height - padding):
    for x in range(padding, image_width - padding):
      image_patch = image[y - padding:y + padding + 1, x - padding:x + padding + 1]
      filtered_image[y, x] = np.sum(image_patch * filter)

  return filtered_image

# 가우시안 피라미드 생성 함수
def gaussian_pyramid(image, scale_num, levels): # scale_num: 0~1
    pyramid_img = [image]
    downsample_num = int(1 // scale_num)
    for _ in range(levels - 1): # 반복 변수를 직접 사용하지 않을 때 관습적 표현
        blur_img = convolution(pyramid_img[-1], gaussian_filter_mask) # 고주파 노이즈 제거
        downsample_img = blur_img[::downsample_num,::downsample_num] # 행,열을 2칸씩 건너 뛰면서 샘플링-->0.5배 다운샘플링
        pyramid_img.append(downsample_img)
    return pyramid_img # 다운샘플된 이미지 리스트 반환

In [ ]:
# 가우시안 피라미드 결과
gp_img = gaussian_pyramid(gray_lena_img, 0.5, 10)

plt.figure(figsize=(50, 50))
for i in range(len(gp_img)):
    plt.subplot(1, 10, i+1)
    plt.imshow(gp_img[i], cmap='gray')
    plt.title(f'Level {i}')
    plt.axis('on')
plt.show()

In [ ]:
print('size of lv0: ', gp_img[0].shape)
print('size of lv1: ', gp_img[1].shape)
print('size of lv2: ', gp_img[2].shape)
print('size of lv3: ', gp_img[3].shape)

plt.imshow(gp_img[0], cmap='gray')
plt.title('Level 0(original)')
plt.axis('on')
plt.show()

plt.imshow(gp_img[1], cmap='gray')
plt.title('Level 1')
plt.axis('on')
plt.show()

plt.imshow(gp_img[2], cmap='gray')
plt.title('Level 2')
plt.axis('on')
plt.show()

plt.imshow(gp_img[3], cmap='gray')
plt.title('Level 3')
plt.axis('on')
plt.show()

In [ ]:
# # Input : image, scaling factor (0<s<1), number of levels
# # Output : image pyramid
# cv2.pyDown 함수 사용 버전
def gaussian_pyramid_cv2_version(image, scale, levels):
    pyramid = [image]
    for _ in range(levels - 1):
        image = cv2.pyrDown(image) # cv2.pyDown 함수 사용
        pyramid.append(image)
    return pyramid

gp_img_cv2_version = gaussian_pyramid_cv2_version(gray_lena_img, 0.5, 10)

plt.figure(figsize=(15, 5))
for i in range(len(gp_img_cv2_version)):
    plt.subplot(1, 10, i+1)
    plt.imshow(gp_img_cv2_version[i], cmap='gray')
    plt.title(f'Level {i}')
    plt.axis('on')
plt.show()

In [ ]:
# prompt: # 2) Design a function to generate Laplacian image pyramid.
# # Input : image, scaling factor (0<s<1), number of levels
# # Output : image pyramid
# cv2함수 사용한 버전

def laplacian_pyramid_cv2_version(image, scale, levels):
    gaussian_pyr = gaussian_pyramid(image, scale, levels)
    laplacian_pyr = []
    for i in range(levels - 1):
        upsampled = cv2.pyrUp(gaussian_pyr[i + 1])
        # Ensure upsampled image has the same size as the original
        rows, cols = gaussian_pyr[i].shape[:2]
        upsampled = cv2.resize(upsampled, (cols, rows))
        laplacian = cv2.subtract(gaussian_pyr[i], upsampled)
        laplacian_pyr.append(laplacian)
    laplacian_pyr.append(gaussian_pyr[-1])  # Add the last Gaussian level
    return laplacian_pyr

lp_img = laplacian_pyramid_cv2_version(gray_lena_img, 0.5, 10)

plt.figure(figsize=(15, 5))
for i in range(len(lp_img)):
    plt.subplot(1, 10, i+1)
    plt.imshow(lp_img[i], cmap='gray')
    plt.title(f'Level {i}')
    plt.axis('on')
plt.show()


In [ ]:
# Laplacian 피라미드는 한 이미지에서 더 낮은 해상도의 이미지를 제거한 것
# 원본 이미지에서 흐릿하고 축소된 이미지를 뺀 것
# L[i] = Gaussian[i] - upsample(Gassian[i+1])
# Gaussian[i+1]은 다운샘플링 되어 있으므로 zero-padding을 해주고 빼야함-->Gaussian[i+1] 업샘플링 진행

# downsample 시킨 비율에 따라 다시 upsampling 해주는 함수
def upsample_func(pre_img, post_img, scale_ratio, g_mask): # 이전 이미지에 비해 어느 정도로 downsample이 되었는지 요소를 넣는다
    """
    pre_img: 다운샘플된 이미지 G[i+1]
    post_img: 기준 이미지 G[i] (해상도 기준)
    scale_ratio: 예: 0.5 (2배 업샘플)
    g_mask: 가우시안 필터 마스크
    """
    scale_num = round(1/scale_ratio) # int보다 안전한 round 사용

    h, w = post_img.shape # upsample 해야할 크기
    upsampled_img = np.zeros((h, w), dtype=np.float32)
    upsampled_img[::scale_num, ::scale_num] = pre_img

    # 가우시안 마스크로 부드럽게 보간 효과 추가
    upsampled_blurred_img = convolution(upsampled_img, gaussian_filter_mask * scale_num)
    return upsampled_blurred_img

# 2배 업샘플링 함수
def double_upsample(img):
    h,w = img.shape
    upsampled_img = np.zeros((h*2, w*2), dtype=np.float32)
    upsampled_img[::2, ::2] = img
    return upsampled_img

def laplacian_pyramid(image, scale, levels):
    g_pyramid = gaussian_pyramid(image, scale, levels) # 가우시안 피라미드 이미지 리스트
    l_pyramid = [] # 라플라시안 피라미드 이미지 리스트

    for i in range(levels - 1):
        # 업샘플된 Gaussian[i+1]을 Gaussian[i]와 같은 크기로 만든다
        # upsampled_img = upsample_func(g_pyramid[i+1], g_pyramid[i], 0.5, gaussian_filter_mask)
        upsampled_img = double_upsample(g_pyramid[i+1])

        # Laplacian[i] = Gaussian[i] - upsample(Gaussian[i+1])
        laplacian_img = g_pyramid[i] - upsampled_img
        l_pyramid.append(laplacian_img)


    l_pyramid.append(g_pyramid[-1]) # 마지막은 그대로

    return l_pyramid


In [ ]:
laplacian_pyramid_img_list = laplacian_pyramid(gray_lena_img, 0.5, levels=10)

plt.figure(figsize=(50, 50))
for i, lap_img in enumerate(laplacian_pyramid_img_list):
    plt.subplot(1, len(laplacian_pyramid_img_list), i+1)
    plt.imshow(lap_img, cmap='gray')
    plt.title(f'Laplacian {i}')
    plt.axis('off')
plt.show()